In [10]:
# Facial Expression Recognition using FER-2013 Dataset 

# Step 1: Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten,
                                     Dense, Dropout, BatchNormalization)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import os
import warnings
warnings.filterwarnings('ignore')

# Step 2: Load and Inspect the Dataset
def load_and_inspect_data(dataset_path):
    """Load the FER-2013 dataset and inspect its structure"""
    try:
        print("Loading FER-2013 dataset...")
        fer_df = pd.read_csv(dataset_path)
        
        print(f"Dataset shape: {fer_df.shape}")
        print(f"Columns: {fer_df.columns.tolist()}")
        print(f"First few rows:")
        print(fer_df.head())
        
        # Check for missing values
        print(f"\nMissing values:\n{fer_df.isnull().sum()}")
        
        # Check emotion distribution
        if 'emotion' in fer_df.columns:
            print(f"\nEmotion distribution:")
            print(fer_df['emotion'].value_counts().sort_index())
        
        return fer_df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

# Update this path to your actual dataset location
dataset_path = 'fer2013.csv' 
fer_df = load_and_inspect_data(dataset_path)

# Step 3: Improved Data Preprocessing
def prepare_data(df):
    """Prepare images and labels from the FER-2013 dataset"""
    try:
        # Convert pixel strings to numpy arrays
        print("Converting pixel data to images...")
        pixel_series = df['pixels'].apply(lambda pixel_str: np.fromstring(pixel_str, sep=' '))
        image_array = np.stack(pixel_series.to_numpy())
        
        # Reshape and normalize
        image_array = image_array.reshape(-1, 48, 48, 1).astype('float32') / 255.0
        
        # Convert labels to one-hot encoding
        label_array = pd.get_dummies(df['emotion']).values
        
        print(f"Images shape: {image_array.shape}")
        print(f"Labels shape: {label_array.shape}")
        
        return image_array, label_array
    except Exception as e:
        print(f"Error in data preparation: {e}")
        return None, None

if fer_df is not None:
    images, labels = prepare_data(fer_df)
else:
    print("Cannot proceed without dataset. Please check the dataset path.")
    exit()

# Step 4: Enhanced Data Visualization
emotion_names = {
    0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy",
    4: "Sad", 5: "Surprise", 6: "Neutral"
}

def visualize_data(images, labels, emotion_names):
    """Visualize sample images and emotion distribution"""
    # Sample images
    plt.figure(figsize=(15, 10))
    
    # Plot sample images
    plt.subplot(2, 2, (1, 2))
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    for i in range(8):
        row, col = i // 4, i % 4
        axes[row, col].imshow(images[i].reshape(48, 48), cmap='gray')
        axes[row, col].set_title(f"Emotion: {emotion_names[np.argmax(labels[i])]}")
        axes[row, col].axis('off')
    plt.suptitle("Sample Images from FER-2013 Dataset")
    plt.tight_layout()
    plt.show()
    
    # Plot emotion distribution
    plt.figure(figsize=(10, 6))
    emotion_counts = [np.sum(np.argmax(labels, axis=1) == i) for i in range(7)]
    plt.bar(range(7), emotion_counts, color='skyblue')
    plt.xlabel('Emotion')
    plt.ylabel('Count')
    plt.title('Distribution of Emotions in Dataset')
    plt.xticks(range(7), [emotion_names[i] for i in range(7)], rotation=45)
    plt.tight_layout()
    plt.show()

visualize_data(images, labels, emotion_names)

# Step 5: Improved Data Splitting
print("Splitting dataset...")
train_images, temp_images, train_labels, temp_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42, stratify=labels
)
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print(f"Training set: {train_images.shape[0]} samples")
print(f"Validation set: {val_images.shape[0]} samples")
print(f"Test set: {test_images.shape[0]} samples")

# Step 6: Enhanced Data Augmentation
data_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2]
)

# Step 7: Improved CNN Model Architecture
def create_cnn_model(input_shape=(48, 48, 1), num_classes=7):
    """Create an improved CNN model for facial expression recognition"""
    model = Sequential([
        # First Convolutional Block
        Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Second Convolutional Block
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Third Convolutional Block
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Fully Connected Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

expression_model = create_cnn_model()

# Step 8: Compile the Model
expression_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
print("Model Architecture:")
expression_model.summary()

# Step 9: Enhanced Training with Callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5, min_lr=0.00001)
]

print("Starting model training...")
training_history = expression_model.fit(
    data_generator.flow(train_images, train_labels, batch_size=32),
    validation_data=(val_images, val_labels),
    epochs=50,
    callbacks=callbacks,
    verbose=1
)

# Step 10: Save the Model
model_save_path = 'fer2013_expression_model.h5'
expression_model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

# Step 11: Comprehensive Model Evaluation
def evaluate_model(model, test_images, test_labels, emotion_names):
    """Evaluate the model and generate comprehensive reports"""
    # Evaluate on test set
    test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    
    # Generate predictions
    predicted_probs = model.predict(test_images)
    predicted_classes = np.argmax(predicted_probs, axis=1)
    true_classes = np.argmax(test_labels, axis=1)
    
    # Classification Report
    print("\nClassification Report:")
    print(classification_report(true_classes, predicted_classes, 
                              target_names=[emotion_names[i] for i in range(7)]))
    
    # Confusion Matrix
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[emotion_names[i] for i in range(7)],
                yticklabels=[emotion_names[i] for i in range(7)])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    
    return test_accuracy, predicted_classes, true_classes

test_accuracy, predicted_classes, true_classes = evaluate_model(
    expression_model, test_images, test_labels, emotion_names
)

# Step 12: Enhanced Visualization of Training History
def plot_training_history(history):
    """Plot training history with improved visualizations"""
    plt.figure(figsize=(15, 5))
    
    # Accuracy plot
    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='red')
    plt.title('Model Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Loss plot
    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='blue')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='red')
    plt.title('Model Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Learning rate plot (if available)
    plt.subplot(1, 3, 3)
    if 'lr' in history.history:
        plt.plot(history.history['lr'], label='Learning Rate', color='green')
        plt.title('Learning Rate Over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Learning Rate')
        plt.legend()
        plt.grid(True)
    else:
        plt.text(0.5, 0.5, 'Learning Rate\nNot Available', 
                ha='center', va='center', transform=plt.gca().transAxes)
    
    plt.tight_layout()
    plt.show()

plot_training_history(training_history)

# Step 13: Additional Analysis and Insights
def analyze_predictions(predicted_classes, true_classes, emotion_names):
    """Analyze prediction patterns and provide insights"""
    print("\n" + "="*50)
    print("ADDITIONAL ANALYSIS")
    print("="*50)
    
    # Per-class accuracy
    print("\nPer-class Accuracy:")
    for i in range(7):
        class_mask = true_classes == i
        if np.sum(class_mask) > 0:
            class_accuracy = np.mean(predicted_classes[class_mask] == true_classes[class_mask])
            print(f"{emotion_names[i]}: {class_accuracy:.4f}")
    
    # Most confused emotions
    cm = confusion_matrix(true_classes, predicted_classes)
    print("\nMost Confused Emotion Pairs:")
    for i in range(7):
        for j in range(7):
            if i != j and cm[i, j] > 0:
                confusion_rate = cm[i, j] / np.sum(cm[i, :])
                if confusion_rate > 0.1:  # Show confusions > 10%
                    print(f"{emotion_names[i]} → {emotion_names[j]}: {confusion_rate:.2%}")

analyze_predictions(predicted_classes, true_classes, emotion_names)

# Step 14: Suggestions for Improvement
print("\n" + "="*50)
print("SUGGESTIONS FOR IMPROVEMENT")
print("="*50)
print("""
1. Transfer Learning: Use pre-trained models like VGG16, ResNet50, or MobileNet
2. Advanced Architectures: Try ResNet, DenseNet, or EfficientNet
3. Ensemble Methods: Combine multiple models for better performance
4. Data Augmentation: Experiment with more sophisticated augmentation techniques
5. Hyperparameter Tuning: Use tools like Keras Tuner or Optuna
6. Real-time Implementation: Integrate with OpenCV for live emotion detection
7. Cross-validation: Use k-fold cross-validation for more robust evaluation
8. Attention Mechanisms: Add attention layers to focus on important facial features
""")

print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
print("Model training and evaluation completed successfully!")

ModuleNotFoundError: No module named 'tensorflow'